In [5]:
import os
import time
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import pandas as pd

def save_progress(completed_brands, filename='completed_brands.csv'):
    """진행 상태를 저장하는 함수"""
    pd.DataFrame(completed_brands, columns=['Brand', 'Page_Number']).to_csv(filename, index=False)

def load_progress(filename='completed_brands.csv'):
    """저장된 진행 상태를 로드하는 함수"""
    try:
        return pd.read_csv(filename).to_dict('records')
    except FileNotFoundError:
        return []

# 웹 드라이버 설정
options = webdriver.ChromeOptions()
options.add_argument('--disable-dev-shm-usage')
options.add_argument("--no-sandbox")
driver = webdriver.Chrome(options=options)
driver.set_window_size(1200, 2000)

# 브랜드명 리스트 로드
df = pd.read_csv('Brand.csv')
brand_names = df['Brand_ENG'].tolist()

# 이미 완료된 브랜드와 페이지 로드
completed_brands = load_progress()

data = []

for brand in tqdm(brand_names, desc='Brands'):
    # 이 브랜드의 마지막으로 크롤링한 페이지 번호 가져오기
    last_page_number = next((item['Page_Number'] for item in completed_brands if item['Brand'] == brand), 1)
    for page_num in range(last_page_number, 101):  # 최대 100페이지까지
        driver.get(f'https://www.musinsa.com/brands/{brand}?category3DepthCodes=&category2DepthCodes=&category1DepthCode=&colorCodes=&startPrice=&endPrice=&exclusiveYn=&includeSoldOut=&saleGoods=&timeSale=&includeKeywords=&sortCode=NEW&tags=&page={page_num}&size=90&listViewType=small&campaignCode=&outletGoods=&plusDelivery=')

        # 현재 페이지에서 상품 처리
        found_item = False  # 상품을 찾았는지 여부를 나타내는 플래그
        for item in range(1, 91):  # 한 페이지에 최대 90개 상품이 있다고 가정
            try:
                # 제품 가격 
                item_price_xpath = f'//*[@id="searchList"]/li[{item}]/div[4]/div[2]/p[3]'
                item_price_element = driver.find_element(By.XPATH, item_price_xpath)
                item_price = item_price_element.text  # 가격 정보 텍스트 수집

                item_element = driver.find_element(By.XPATH, f'//*[@id="searchList"]/li[{item}]/div[4]/div[2]/p[2]/a').click()
                
                try:
                    see_button = driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[2]/div[1]/div[4]/button')
                    see_button.click()
                except NoSuchElementException:
                    pass  # 팝업 창이 없는 경우 그냥 지나감

                # 상품 상세 정보 수집
                item_category = driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[1]/div[1]/a[1]').text
                item_img = driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[1]/div[1]/div/img').get_attribute('src')
                item_name = driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[1]/div[3]/div[2]/h3').text
                item_num = driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[2]/div[2]/ul/li[1]/div[2]/span[2]').text
                item_season = driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[2]/div[2]/ul/li[2]/div[2]/span[1]').text
                item_sex = driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[2]/div[2]/ul/li[2]/div[2]/span[3]').text

                # 수집된 데이터 추가
                data.append([item_category, brand, page_num, item_img, item_name, item_num, item_season, item_sex, item_price])
                
                driver.back()  # 상품 목록 페이지로 돌아가기
                time.sleep(1.5)  # 페이지 로딩 대기
                found_item = True  # 상품을 찾음
            except NoSuchElementException:
                print(f"Item not found in the list, item number: {item}")
                # 다음 제품으로 넘어가기
                continue

        # 현재 페이지에서 상품을 찾지 못한 경우에는 다음 브랜드로 넘어감
        if not found_item:
            break

        # 다음 페이지로 이동
        try:
            next_page_button = driver.find_element(By.XPATH, '//*[@id="searchPaging"]/div/a[@class="btn_page btn_next"]')
            next_page_button.click()
            
        except NoSuchElementException:
            print("Next page not found")
            # 다음 페이지로 넘어가지 않으면 다음 브랜드로 넘어가기
            break

        # 마지막 페이지까지 크롤링 완료한 브랜드와 페이지 번호 저장
        completed_brands.append({'Brand': brand, 'Page_Number': page_num})
        save_progress(completed_brands)

        # 데이터를 중간 저장합니다.
        new_data = pd.DataFrame(data, columns=['Item_Category', 'Brand', 'Page_Number', 'Item_Image', 'Item_Name', 'Item_Number', 'Item_Season', 'Item_Sex', 'Item_Price'])
        if not os.path.exists('Items_Info.csv'):
            new_data.to_csv('Items_Info.csv', index=False)
        else:
            new_data.to_csv('Items_Info.csv', mode='a', header=False, index=False)
        data.clear()  # 다음 반복을 위해 데이터 리스트를 비웁니다.

driver.quit()


Brands:   0%|          | 0/2929 [00:00<?, ?it/s]

Item not found or page load timeout, item number: 1
Item not found or page load timeout, item number: 2


Brands:   0%|          | 0/2929 [00:32<?, ?it/s]


KeyboardInterrupt: 

In [1]:
import os
import time
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import pandas as pd

def save_progress(completed_brands, filename='completed_brands.csv'):
    pd.DataFrame(completed_brands, columns=['Brand', 'Page_Number']).to_csv(filename, index=False)

def load_progress(filename='completed_brands.csv'):
    try:
        return pd.read_csv(filename).to_dict('records')
    except FileNotFoundError:
        return []

options = webdriver.ChromeOptions()
options.add_argument('--disable-dev-shm-usage')
options.add_argument("--no-sandbox")
driver = webdriver.Chrome(options=options)
driver.set_window_size(2500, 2500)

df = pd.read_csv('Brand.csv')
brand_names = df['Brand_ENG'].tolist()

completed_brands = load_progress()

data = []

for brand in tqdm(brand_names, desc='Brands'):
    last_page_number = next((item['Page_Number'] for item in completed_brands if item['Brand'] == brand), 1)
    for page_num in range(last_page_number, 101):
        driver.get(f'https://www.musinsa.com/brands/{brand}?page={page_num}&size=90')

        try:
            WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.XPATH, '//*[@id="result-none-area"]/p')))
            no_items_text = driver.find_element(By.XPATH, '//*[@id="result-none-area"]/p').text
            if "등록된 상품이 없습니다." in no_items_text:
                print(f"No items for brand {brand} on page {page_num}. Moving to next brand.")
                break
        except TimeoutException:
            pass

        found_item = False
        for item in range(1, 91):
            try:
                item_price_xpath = f'//*[@id="searchList"]/li[{item}]/div[4]/div[2]/p[3]'
                item_price_element = driver.find_element(By.XPATH, item_price_xpath)
                item_price = item_price_element.text  

                item_element = driver.find_element(By.XPATH, f'//*[@id="searchList"]/li[{item}]/div[4]/div[2]/p[2]/a').click()
                
                WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="root"]/div[1]/div[1]/div[3]/div[2]/h3')))
                item_details = {
                    "item_category": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[1]/div[1]/a[1]').text,
                    "item_img": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[1]/div[1]/div/img').get_attribute('src'),
                    "item_name": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[1]/div[3]/div[2]/h3').text,
                    "item_num": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[2]/div[2]/ul/li[1]/div[2]/span[2]').text,
                    "item_season": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[2]/div[2]/ul/li[2]/div[2]/span[1]').text,
                    "item_sex": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[2]/div[2]/ul/li[2]/div[2]/span[3]').text,
                    "item_price": item_price
                }
                data.append(item_details)
                print(item_details)
                driver.back()
                WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, item_price_xpath)))
                found_item = True
            except (NoSuchElementException, TimeoutException):
                print(f"Item not found or page load timeout, item number: {item}")
                continue

        if not found_item:
            break

        completed_brands.append({'Brand': brand, 'Page_Number': page_num})
        save_progress(completed_brands)

        if data:
            new_data = pd.DataFrame(data)
            if not os.path.exists('Items_Info.csv'):
                new_data.to_csv('Items_Info.csv', index=False)
            else:
                new_data.to_csv('Items_Info.csv', mode='a', header=False, index=False)
            data.clear()

driver.quit()


Brands:   0%|          | 0/2929 [00:00<?, ?it/s]

Item not found or page load timeout, item number: 1
Item not found or page load timeout, item number: 2
Item not found or page load timeout, item number: 3
Item not found or page load timeout, item number: 4
Item not found or page load timeout, item number: 5
Item not found or page load timeout, item number: 6
Item not found or page load timeout, item number: 7
Item not found or page load timeout, item number: 8
Item not found or page load timeout, item number: 9
Item not found or page load timeout, item number: 10
Item not found or page load timeout, item number: 11
Item not found or page load timeout, item number: 12
Item not found or page load timeout, item number: 13
Item not found or page load timeout, item number: 14
Item not found or page load timeout, item number: 15
Item not found or page load timeout, item number: 16
Item not found or page load timeout, item number: 17
Item not found or page load timeout, item number: 18
Item not found or page load timeout, item number: 19
It

Brands:   0%|          | 1/2929 [00:08<6:54:18,  8.49s/it]

Item not found or page load timeout, item number: 90
Item not found or page load timeout, item number: 1
Item not found or page load timeout, item number: 2
Item not found or page load timeout, item number: 3
Item not found or page load timeout, item number: 4
Item not found or page load timeout, item number: 5
Item not found or page load timeout, item number: 6
Item not found or page load timeout, item number: 7
Item not found or page load timeout, item number: 8
Item not found or page load timeout, item number: 9
Item not found or page load timeout, item number: 10
Item not found or page load timeout, item number: 11
Item not found or page load timeout, item number: 12
Item not found or page load timeout, item number: 13
Item not found or page load timeout, item number: 14
Item not found or page load timeout, item number: 15
Item not found or page load timeout, item number: 16
Item not found or page load timeout, item number: 17
Item not found or page load timeout, item number: 18
It

Brands:   0%|          | 2/2929 [00:14<5:42:08,  7.01s/it]

Item not found or page load timeout, item number: 88
Item not found or page load timeout, item number: 89
Item not found or page load timeout, item number: 90
Item not found or page load timeout, item number: 1


: 

In [5]:
import os
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException

def setup_driver():
    options = webdriver.ChromeOptions()
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument("--no-sandbox")
    driver = webdriver.Chrome(options=options)
    driver.set_window_size(1800, 1800)
    return driver

def load_progress(filename='completed_brands.csv'):
    try:
        return pd.read_csv(filename).to_dict('records')
    except FileNotFoundError:
        return []

def save_progress(completed_brands, filename='completed_brands.csv'):
    pd.DataFrame(completed_brands, columns=['Brand', 'Page_Number']).to_csv(filename, index=False)

def scrape_brand_data(driver, brand, completed_brands, start_page=1, end_page=100):
    for page_num in range(start_page, end_page + 1):
        page_url = f'https://www.musinsa.com/brands/{brand}?page={page_num}&size=90'
        driver.get(page_url)

        if is_page_empty(driver):
            break

        data = []
        for item in range(1, 3):
            driver.get(page_url)
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, f'//*[@id="searchList"]/li[{item}]')))

            try:
                item_details = scrape_item(driver, item)
                if item_details:
                    data.append(item_details)
                    print(item_details)
            except Exception as e:
                print(f"Error scraping item {item} on page {page_num} for {brand}: {e}")
                continue

        save_data(data, 'Items_Info.csv')
        data.clear()

        completed_brands.append({'Brand': brand, 'Page_Number': page_num})
        save_progress(completed_brands)

def is_page_empty(driver):
    try:
        # 페이지의 요소가 로드될 때까지 최대 10초 대기
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="searchList"]/li[1]')))
        return False  # 제품 목록이 로드되면 False 반환
    except TimeoutException:
        # 제품 목록이 로드되지 않고, "등록된 상품이 없습니다" 메시지가 나타나면 True 반환
        try:
            no_items_text = driver.find_element(By.XPATH, '//*[@id="result-none-area"]/p').text
            if "등록된 상품이 없습니다" in no_items_text:
                return True
        except NoSuchElementException:
            pass
    return False  # "등록된 상품이 없습니다" 메시지가 없으면 False 반환

def scrape_item(driver, item_position, return_url):
    try:
        # 제품 상세 페이지로 이동하기 전 현재 페이지의 URL을 저장합니다.
        current_page_url = return_url

        # 제품 상세 정보 페이지로 이동합니다.
        item_link_xpath = f'//*[@id="searchList"]/li[{item_position}]/div[4]/div[2]/p[2]/a'
        item_link = driver.find_element(By.XPATH, item_link_xpath).get_attribute('href')
        driver.get(item_link)

        WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="root"]/div[1]/div[1]/div[3]/div[2]/h3')))
        item_details = {
            "item_category": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[1]/div[1]/a[1]').text,
            "item_img": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[1]/div[1]/div/img').get_attribute('src'),
            "item_name": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[1]/div[3]/div[2]/h3').text,
            "item_num": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[2]/div[2]/ul/li[1]/div[2]/span[2]').text,
            "item_season": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[2]/div[2]/ul/li[2]/div[2]/span[1]').text,
            "item_sex": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[2]/div[2]/ul/li[2]/div[2]/span[3]').text,
            "item_price": item_price
        }
        driver.get(current_page_url)
        return item_details
    

    except Exception as e:
        print(f"An error occurred while scraping the item: {e}")
        driver.get(current_page_url)  # 오류 발생 시, 원래 페이지로 돌아갑니다.
        return None
def save_data(data, filename):
    new_data = pd.DataFrame(data)
    if not os.path.exists(filename):
        new_data.to_csv(filename, index=False)
    else:
        new_data.to_csv(filename, mode='a', header=False, index=False)

def main():
    driver = setup_driver()
    df = pd.read_csv('Brand.csv')
    brand_names = df['Brand_ENG'].tolist()
    completed_brands = load_progress()

    for brand in brand_names:
        last_completed_page = next((item['Page_Number'] for item in completed_brands if item['Brand'] == brand), 0)
        start_page = last_completed_page + 1
        print(f"Starting scraping for brand: {brand} from page {start_page}")
        scrape_brand_data(driver, brand, completed_brands, start_page=start_page)

        if start_page > 1:  # If it started from a page beyond 1, it means some work was done
            completed_brands.append({'Brand': brand, 'Page_Number': start_page})
            save_progress(completed_brands)

    driver.quit()
if __name__ == "__main__":
    main()    

Starting scraping for brand: MUSINSASTANDARD from page 2
Error scraping item 1 on page 2 for MUSINSASTANDARD: scrape_item() missing 1 required positional argument: 'return_url'
Error scraping item 2 on page 2 for MUSINSASTANDARD: scrape_item() missing 1 required positional argument: 'return_url'
Error scraping item 1 on page 3 for MUSINSASTANDARD: scrape_item() missing 1 required positional argument: 'return_url'
Error scraping item 2 on page 3 for MUSINSASTANDARD: scrape_item() missing 1 required positional argument: 'return_url'


: 

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
import time

def scrape_product_info(driver, item):
    try:
        item_price_xpath = f'//*[@id="searchList"]/li[{item}]/div[4]/div[2]/p[3]'
        item_price_element = driver.find_element(By.XPATH, item_price_xpath)
        item_price = item_price_element.text

        item_element_xpath = f'//*[@id="searchList"]/li[{item}]/div[4]/div[2]/p[2]/a'
        item_element = driver.find_element(By.XPATH, item_element_xpath)
        driver.execute_script("arguments[0].click();", item_element)
        time.sleep(0.5)

        try:
            see_button = driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[2]/div[1]/div[4]/button')
            see_button.click()
            time.sleep(0.5)
        except NoSuchElementException:
            pass

        item_details = {
            "item_category": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[1]/div[1]/a[1]').text,
            "item_img": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[1]/div[1]/div/img').get_attribute('src'),
            "item_name": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[1]/div[3]/div[2]/h3').text,
            "item_num": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[2]/div[2]/ul/li[1]/div[2]/span[2]').text,
            "item_season": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[2]/div[2]/ul/li[2]/div[2]/span[1]').text,
            "item_sex": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[2]/div[2]/ul/li[2]/div[2]/span[3]').text,
            "item_price": item_price
        }

        driver.back()
        time.sleep(1)
        return item_details
    except NoSuchElementException:
        print(f"Item not found in the list, item number: {item}")
        return None
    except Exception as e:
        print(f"An error occurred while scraping item {item}: {e}")
        return None

def setup_driver():
    options = webdriver.ChromeOptions()
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument("--no-sandbox")
    driver = webdriver.Chrome(options=options)
    driver.set_window_size(1200, 2000)
    return driver

def main():
    driver = setup_driver()
    df = pd.read_csv('Brand.csv')
    brand_names = df['Brand_ENG'].tolist()

    data = []

    for brand in brand_names:
        driver.get(f'https://www.musinsa.com/brands/{brand}')

        for item in range(1, 3):  # 예시로 첫 두 제품만 대상으로 합니다.
            product_info = scrape_product_info(driver, item)
            if product_info:
                data.append([brand, product_info['item_category'], product_info['item_img'], product_info['item_name'], product_info['item_num'], product_info['item_season'], product_info['item_sex'], product_info['item_price']])

    df_results = pd.DataFrame(data, columns=['Brand', 'Category', 'Image', 'Name', 'Number', 'Season', 'Sex', 'Price'])
    df_results.to_csv('Scraped_Products_Info.csv', index=False)

    driver.quit()

if __name__ == "__main__":
    main()

Item not found in the list, item number: 1
Item not found in the list, item number: 2
Item not found in the list, item number: 1
Item not found in the list, item number: 2
Item not found in the list, item number: 1
Item not found in the list, item number: 2


: 

페이지마다 반은 못가져옴..

In [1]:
import os
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from tqdm import tqdm

current_div = 4  # 초기 div 값 설정

def setup_driver():
    options = webdriver.ChromeOptions()
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument("--no-sandbox")
    driver = webdriver.Chrome(options=options)
    driver.set_window_size(1800, 1800)
    return driver

def load_progress(filename='completed_brands.csv'):
    try:
        return pd.read_csv(filename).to_dict('records')
    except FileNotFoundError:
        return []

def save_progress(completed_brands, filename='completed_brands.csv'):
    pd.DataFrame(completed_brands, columns=['Brand', 'Page_Number']).to_csv(filename, index=False)

def is_page_empty(driver):
    try:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="searchList"]/li[1]')))
        return False
    except TimeoutException:
        try:
            no_items_text = driver.find_element(By.XPATH, '//*[@id="result-none-area"]/p').text
            if "등록된 상품이 없습니다" in no_items_text:
                return True
        except NoSuchElementException:
            pass
        return False

def scrape_product_info(driver, item, div):
    try:
        time.sleep(1)
        item_price_xpath = f'//*[@id="searchList"]/li[{item}]/div[{div}]/div[2]/p[3]'

        try:
            item_price_element = driver.find_element(By.XPATH, item_price_xpath)
        except NoSuchElementException:
            # div 값을 변경하여 재시도
            div = 3 if div == 4 else 4
            time.sleep(1)
            item_price_xpath = f'//*[@id="searchList"]/li[{item}]/div[{div}]/div[2]/p[3]'
            item_price_element = driver.find_element(By.XPATH, item_price_xpath)

        item_price = item_price_element.text

        item_element_xpath = f'//*[@id="searchList"]/li[{item}]/div[{div}]/div[2]/p[2]/a'
        item_element = driver.find_element(By.XPATH, item_element_xpath)
        driver.execute_script("arguments[0].click();", item_element)
        time.sleep(1)

        try:
            see_button = driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[2]/div[1]/div[4]/button')
            see_button.click()
            time.sleep(1)
        except NoSuchElementException:
            pass

        item_details = {
            "item_category": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[1]/div[1]/a[1]').text,
            "item_img": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[1]/div[1]/div/img').get_attribute('src'),
            "item_name": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[1]/div[3]/div[2]/h3').text,
            "item_num": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[2]/div[2]/ul/li[1]/div[2]/span[2]').text,
            "item_season": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[2]/div[2]/ul/li[2]/div[2]/span[1]').text,
            "item_sex": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[2]/div[2]/ul/li[2]/div[2]/span[3]').text,
            "item_price": item_price
        }

        driver.back()
        time.sleep(1)
        return item_details, div  # 성공 시 아이템 정보와 현재 div 반환

    except Exception as e:
        print(f"An error occurred: {e}")
        return None, div

def save_data(data, filename):
    if data:
        new_data = pd.DataFrame(data)
        if not os.path.exists(filename):
            new_data.to_csv(filename, index=False)
        else:
            new_data.to_csv(filename, mode='a', header=False, index=False)
    else:
        print("No data to save.")

def scrape_brand_data(driver, brand, completed_brands, start_page=1, end_page=100):
    global current_div  # 전역 변수 사용 선언

    for page_num in range(start_page, end_page + 1):
        page_url = f'https://www.musinsa.com/brands/{brand}?page={page_num}&size=90'
        driver.get(page_url)

        if is_page_empty(driver):
            print(f"No items found for brand {brand} on page {page_num}.")
            break

        data = []
        for item in tqdm(range(1, 91), desc=f"Scraping {brand} - Page {page_num}"):
            item_details, new_div = scrape_product_info(driver, item, current_div)
            current_div = new_div  # 전역 변수 current_div를 업데이트
            if item_details:
                data.append(item_details)
                
            else:  
                print(f"No item found at position {item} for brand {brand} on page {page_num}.")

        save_data(data, 'Items_Info.csv') 
        data.clear()

        completed_brands.append({'Brand': brand, 'Page_Number': page_num}) 
        save_progress(completed_brands)

def main():
    driver = setup_driver()
    df = pd.read_csv('Brand.csv')
    brand_names = df['Brand_ENG'].tolist()
    completed_brands = load_progress()

    for brand in tqdm(brand_names, desc="Overall Progress"):
        last_completed_page = next((item['Page_Number'] for item in completed_brands if item['Brand'] == brand), 0)
        start_page = last_completed_page + 1
        print(f"Starting scraping for brand: {brand} from page {start_page}")
        scrape_brand_data(driver, brand, completed_brands, start_page=start_page)

    driver.quit()

if __name__ == "__main__":
    main()

Overall Progress:   0%|          | 0/2929 [00:00<?, ?it/s]

Starting scraping for brand: MUSINSASTANDARD from page 1


An error occurred: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="root"]/div[1]/div[2]/div[2]/div[2]/ul/li[2]/div[2]/span[3]"}
  (Session info: chrome=122.0.6261.129); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF799F3AD02+56930]
	(No symbol) [0x00007FF799EAF602]
	(No symbol) [0x00007FF799D642E5]
	(No symbol) [0x00007FF799DA98ED]
	(No symbol) [0x00007FF799DA9A2C]
	(No symbol) [0x00007FF799DEA967]
	(No symbol) [0x00007FF799DCBCDF]
	(No symbol) [0x00007FF799DE81E2]
	(No symbol) [0x00007FF799DCBA43]
	(No symbol) [0x00007FF799D9D438]
	(No symbol) [0x00007FF799D9E4D1]
	GetHandleVerifier [0x00007FF79A2B6F8D+3711213]
	GetHandleVerifier [0x00007FF79A3104CD+4077101]
	GetHandleVerifier [0x00007FF79A30865F+4044735]
	GetHandleVerifier [0x00007FF799FD9736+706710]
	(No symbol) [0x00007FF799EBB8DF]
	(No symbol) [0x0000

An error occurred: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="searchList"]/li[43]/div[3]/div[2]/p[3]"}
  (Session info: chrome=122.0.6261.129); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF799F3AD02+56930]
	(No symbol) [0x00007FF799EAF602]
	(No symbol) [0x00007FF799D642E5]
	(No symbol) [0x00007FF799DA98ED]
	(No symbol) [0x00007FF799DA9A2C]
	(No symbol) [0x00007FF799DEA967]
	(No symbol) [0x00007FF799DCBCDF]
	(No symbol) [0x00007FF799DE81E2]
	(No symbol) [0x00007FF799DCBA43]
	(No symbol) [0x00007FF799D9D438]
	(No symbol) [0x00007FF799D9E4D1]
	GetHandleVerifier [0x00007FF79A2B6F8D+3711213]
	GetHandleVerifier [0x00007FF79A3104CD+4077101]
	GetHandleVerifier [0x00007FF79A30865F+4044735]
	GetHandleVerifier [0x00007FF799FD9736+706710]
	(No symbol) [0x00007FF799EBB8DF]
	(No symbol) [0x00007FF799EB6AC4]
	(No s

An error occurred: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="searchList"]/li[44]/div[4]/div[2]/p[3]"}
  (Session info: chrome=122.0.6261.129); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF799F3AD02+56930]
	(No symbol) [0x00007FF799EAF602]
	(No symbol) [0x00007FF799D642E5]
	(No symbol) [0x00007FF799DA98ED]
	(No symbol) [0x00007FF799DA9A2C]
	(No symbol) [0x00007FF799DEA967]
	(No symbol) [0x00007FF799DCBCDF]
	(No symbol) [0x00007FF799DE81E2]
	(No symbol) [0x00007FF799DCBA43]
	(No symbol) [0x00007FF799D9D438]
	(No symbol) [0x00007FF799D9E4D1]
	GetHandleVerifier [0x00007FF79A2B6F8D+3711213]
	GetHandleVerifier [0x00007FF79A3104CD+4077101]
	GetHandleVerifier [0x00007FF79A30865F+4044735]
	GetHandleVerifier [0x00007FF799FD9736+706710]
	(No symbol) [0x00007FF799EBB8DF]
	(No symbol) [0x00007FF799EB6AC4]
	(No s

...

An error occurred: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="searchList"]/li[75]/div[3]/div[2]/p[3]"}
  (Session info: chrome=122.0.6261.129); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF799F3AD02+56930]
	(No symbol) [0x00007FF799EAF602]
	(No symbol) [0x00007FF799D642E5]
	(No symbol) [0x00007FF799DA98ED]
	(No symbol) [0x00007FF799DA9A2C]
	(No symbol) [0x00007FF799DEA967]
	(No symbol) [0x00007FF799DCBCDF]
	(No symbol) [0x00007FF799DE81E2]
	(No symbol) [0x00007FF799DCBA43]
	(No symbol) [0x00007FF799D9D438]
	(No symbol) [0x00007FF799D9E4D1]
	GetHandleVerifier [0x00007FF79A2B6F8D+3711213]
	GetHandleVerifier [0x00007FF79A3104CD+4077101]
	GetHandleVerifier [0x00007FF79A30865F+4044735]
	GetHandleVerifier [0x00007FF799FD9736+706710]
	(No symbol) [0x00007FF799EBB8DF]
	(No symbol) [0x00007FF799EB6AC4]
	(No s

An error occurred: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="searchList"]/li[76]/div[4]/div[2]/p[3]"}
  (Session info: chrome=122.0.6261.129); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF799F3AD02+56930]
	(No symbol) [0x00007FF799EAF602]
	(No symbol) [0x00007FF799D642E5]
	(No symbol) [0x00007FF799DA98ED]
	(No symbol) [0x00007FF799DA9A2C]
	(No symbol) [0x00007FF799DEA967]
	(No symbol) [0x00007FF799DCBCDF]
	(No symbol) [0x00007FF799DE81E2]
	(No symbol) [0x00007FF799DCBA43]
	(No symbol) [0x00007FF799D9D438]
	(No symbol) [0x00007FF799D9E4D1]
	GetHandleVerifier [0x00007FF79A2B6F8D+3711213]
	GetHandleVerifier [0x00007FF79A3104CD+4077101]
	GetHandleVerifier [0x00007FF79A30865F+4044735]
	GetHandleVerifier [0x00007FF799FD9736+706710]
	(No symbol) [0x00007FF799EBB8DF]
	(No symbol) [0x00007FF799EB6AC4]
	(No s

An error occurred: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="searchList"]/li[77]/div[3]/div[2]/p[3]"}
  (Session info: chrome=122.0.6261.129); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF799F3AD02+56930]
	(No symbol) [0x00007FF799EAF602]
	(No symbol) [0x00007FF799D642E5]
	(No symbol) [0x00007FF799DA98ED]
	(No symbol) [0x00007FF799DA9A2C]
	(No symbol) [0x00007FF799DEA967]
	(No symbol) [0x00007FF799DCBCDF]
	(No symbol) [0x00007FF799DE81E2]
	(No symbol) [0x00007FF799DCBA43]
	(No symbol) [0x00007FF799D9D438]
	(No symbol) [0x00007FF799D9E4D1]
	GetHandleVerifier [0x00007FF79A2B6F8D+3711213]
	GetHandleVerifier [0x00007FF79A3104CD+4077101]
	GetHandleVerifier [0x00007FF79A30865F+4044735]
	GetHandleVerifier [0x00007FF799FD9736+706710]
	(No symbol) [0x00007FF799EBB8DF]
	(No symbol) [0x00007FF799EB6AC4]
	(No s

An error occurred: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="searchList"]/li[78]/div[4]/div[2]/p[3]"}
  (Session info: chrome=122.0.6261.129); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF799F3AD02+56930]
	(No symbol) [0x00007FF799EAF602]
	(No symbol) [0x00007FF799D642E5]
	(No symbol) [0x00007FF799DA98ED]
	(No symbol) [0x00007FF799DA9A2C]
	(No symbol) [0x00007FF799DEA967]
	(No symbol) [0x00007FF799DCBCDF]
	(No symbol) [0x00007FF799DE81E2]
	(No symbol) [0x00007FF799DCBA43]
	(No symbol) [0x00007FF799D9D438]
	(No symbol) [0x00007FF799D9E4D1]
	GetHandleVerifier [0x00007FF79A2B6F8D+3711213]
	GetHandleVerifier [0x00007FF79A3104CD+4077101]
	GetHandleVerifier [0x00007FF79A30865F+4044735]
	GetHandleVerifier [0x00007FF799FD9736+706710]
	(No symbol) [0x00007FF799EBB8DF]
	(No symbol) [0x00007FF799EB6AC4]
	(No s

: 

In [ ]:
//*[@id="searchList"]/li[4]/div[4]/div[2]/p[3]
//*[@id="searchList"]/li[44]/div[4]/div[2]/p[3]
//*[@id="searchList"]/li[45]/div[3]/div[2]/p[3]

In [1]:
import os
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from tqdm import tqdm

current_div = 4  # 초기 div 값 설정

def setup_driver():
    options = webdriver.ChromeOptions()
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument("--no-sandbox")
    driver = webdriver.Chrome(options=options)
    driver.set_window_size(1800, 1800)
    return driver

def load_progress(filename='completed_brands.csv'):
    try:
        return pd.read_csv(filename).to_dict('records')
    except FileNotFoundError:
        return []

def save_progress(completed_brands, filename='completed_brands.csv'):
    pd.DataFrame(completed_brands, columns=['Brand', 'Page_Number']).to_csv(filename, index=False)

def is_page_empty(driver):
    try:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="searchList"]/li[1]')))
        return False
    except TimeoutException:
        try:
            no_items_text = driver.find_element(By.XPATH, '//*[@id="result-none-area"]/p').text
            if "등록된 상품이 없습니다" in no_items_text:
                return True
        except NoSuchElementException:
            pass
        return False

def scrape_product_info(driver, item, div, brand, page_num):  # brand 및 page_num 변수 추가
    try:
        item_price_xpath = f'//*[@id="searchList"]/li[{item}]/div[{div}]/div[2]/p[3]'
        item_price_element = driver.find_element(By.XPATH, item_price_xpath)
        item_price = item_price_element.text
        item_element_xpath = f'//*[@id="searchList"]/li[{item}]/div[{div}]/div[1]/a/img'

        driver.find_element(By.XPATH, item_element_xpath).click()
        
        time.sleep(0.5)

        try:
            see_button = driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[2]/div[1]/div[4]/button')
            see_button.click()
            time.sleep(0.5)
        except NoSuchElementException:
            pass

        # item_details = {
        #     "item_category": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[1]/div[1]/a[1]').text,
        #     "item_img": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[1]/div[1]/div/img').get_attribute('src'),
        #     "item_name": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[1]/div[3]/div[2]/h3').text,
        #     "item_num": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[2]/div[2]/ul/li[1]/div[2]/span[2]').text,
        #     "item_season": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[2]/div[2]/ul/li[2]/div[2]/span[1]').text,
        #     "item_sex": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[2]/div[2]/ul/li[2]/div[2]/span[3]').text,
        #     "item_price": item_price
        # }
        item_details = {
            "item_category": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[1]/div[1]/a[1]').text,
            "item_img": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[1]/div[1]/div/img').get_attribute('src'),
            "item_name": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[1]/div[3]/div[2]/h3').text,
            "item_info": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[2]/div[2]/ul').text,
            "item_price": item_price
        }

        driver.back()
        time.sleep(1)
        return item_details, div  # 성공 시 아이템 정보와 현재 div 반환

    except NoSuchElementException:
        print(f"No item found at position {item} for brand {brand} on page {page_num}.")
        return None, div

    except Exception as e:
        print(f"An error occurred: {e}")
        return None, div


def save_data(data, filename):
    if data:
        new_data = pd.DataFrame(data)
        if not os.path.exists(filename):
            new_data.to_csv(filename, index=False)
        else:
            new_data.to_csv(filename, mode='a', header=False, index=False)
    else:
        print("No data to save.")

def scrape_brand_data(driver, brand, completed_brands, start_page=1, end_page=100):
    global current_div  # 전역 변수 사용 선언

    for page_num in range(start_page, end_page + 1):
        page_url = f'https://www.musinsa.com/brands/{brand}?page={page_num}&size=90'
        driver.get(page_url)

        if is_page_empty(driver):
            print(f"No items found for brand {brand} on page {page_num}.")
            break

        data = []
        for item in tqdm(range(1, 91), desc=f"Scraping {brand} - Page {page_num}"):
            item_details, new_div = scrape_product_info(driver, item, current_div, brand, page_num)  # brand 및 page_num 변수 전달
            current_div = new_div  # 전역 변수 current_div를 업데이트
            if item_details:
                data.append(item_details)
            else:  
                print(f"No item found at position {item} for brand {brand} on page {page_num}.")

        save_data(data, 'Items_Info.csv') 
        data.clear()

        completed_brands.append({'Brand': brand, 'Page_Number': page_num}) 
        save_progress(completed_brands)

def main():
    driver = setup_driver()
    df = pd.read_csv('Brand.csv')
    brand_names = df['Brand_ENG'].tolist()
    completed_brands = load_progress()

    for brand in tqdm(brand_names, desc="Overall Progress"):
        last_completed_page = next((item['Page_Number'] for item in completed_brands if item['Brand'] == brand), 0)
        start_page = last_completed_page + 1
        print(f"Starting scraping for brand: {brand} from page {start_page}")
        scrape_brand_data(driver, brand, completed_brands, start_page=start_page)

    driver.quit()

if __name__ == "__main__":
    main()

Overall Progress:   0%|          | 0/2929 [00:00<?, ?it/s]

Starting scraping for brand: MUSINSASTANDARD from page 1


가격: 99,900원
상세: {'item_category': '아우터', 'item_img': 'https://image.msscdn.net/images/goods_img/20240105/3779126/3779126_17103956089248_500.jpg', 'item_name': '오버사이즈 데님 블루종 재킷 [워시드 블랙]', 'item_info': '브랜드\n품번\nMUSINSA STANDARD\nMMDNJ101-WC\n시즌\n성별\n2024 S/S\n남성\n조회수(1개월)\n1.2천 회 이상\n좋아요\n74', 'item_price': '99,900원'}


가격: 69,900원
상세: {'item_category': '아우터', 'item_img': 'https://image.msscdn.net/images/goods_img/20231206/3746106/3746106_17103952311753_500.jpg', 'item_name': '쿨 터치 에센셜 가디건 [스카이 블루]', 'item_info': '브랜드\n품번\nMUSINSA STANDARD\nMMCWC106-SB\n성별\n남성\n조회수(1개월)\n150 회 이상\n좋아요\n12', 'item_price': '69,900원'}


가격: 69,900원
상세: {'item_category': '아우터', 'item_img': 'https://image.msscdn.net/images/goods_img/20231206/3746105/3746105_17103953319493_500.jpg', 'item_name': '쿨 터치 에센셜 가디건 [더스티 그린]', 'item_info': '브랜드\n품번\nMUSINSA STANDARD\nMMCWC106-UG\n성별\n남성\n조회수(1개월)\n150 회 이상\n좋아요\n8', 'item_price': '69,900원'}


: 

In [1]:
import os
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from tqdm import tqdm

current_div = 4  # 초기 div 값 설정

def setup_driver():
    options = webdriver.ChromeOptions()
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument("--no-sandbox")
    driver = webdriver.Chrome(options=options)
    driver.set_window_size(1800, 1800)
    return driver

def load_progress(filename='completed_brands.csv'):
    try:
        return pd.read_csv(filename).to_dict('records')
    except FileNotFoundError:
        return []

def save_progress(completed_brands, filename='completed_brands.csv'):
    pd.DataFrame(completed_brands, columns=['Brand', 'Page_Number']).to_csv(filename, index=False)

def is_page_empty(driver):
    try:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="searchList"]/li[1]')))
        return False
    except TimeoutException:
        try:
            no_items_text = driver.find_element(By.XPATH, '//*[@id="result-none-area"]/p').text
            if "등록된 상품이 없습니다" in no_items_text:
                return True
        except NoSuchElementException:
            
            pass
        return False

def scrape_product_info(driver, item, div, brand, page_num):  # brand 및 page_num 변수 추가
    try:

        try:
            item_price_xpath = f'//*[@id="searchList"]/li[{item}]/div[{div}]/div[2]/p[3]'
            item_price_element = driver.find_element(By.XPATH, item_price_xpath)
            item_price = item_price_element.text
            item_element_xpath = f'//*[@id="searchList"]/li[{item}]/div[{div}]/div[1]/a/img'
        except Exception as e:
            if div == 4:
                div = 3
                item_price_xpath = f'//*[@id="searchList"]/li[{item}]/div[{div}]/div[2]/p[3]'
                item_price_element = driver.find_element(By.XPATH, item_price_xpath)
                item_price = item_price_element.text
                item_element_xpath = f'//*[@id="searchList"]/li[{item}]/div[{div}]/div[1]/a/img'
                pass
        
        driver.find_element(By.XPATH, item_element_xpath).click()
      
        try:
            item_details = {
            "item_category": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[1]/div[1]/a[1]').text,
            "item_img": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[1]/div[1]/div/img').get_attribute('src'),
            "item_name": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[1]/div[3]/div[2]/h3').text,
            "item_num": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[2]/div[2]/ul/li[1]/div[2]/span[2]').text,
            "item_season": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[2]/div[2]/ul/li[2]/div[2]/span[1]').text,
            "item_sex": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[2]/div[2]/ul/li[2]/div[2]/span[3]').text,
            "item_price": item_price
        }
            
            driver.back()
        except Exception as e:
            item_details = {
            "item_category": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[1]/div[1]/a[1]').text,
            "item_img": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[1]/div[1]/div/img').get_attribute('src'),
            "item_name": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[1]/div[3]/div[2]/h3').text,
            "item_num": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[2]/div[2]/ul/li[1]/div[2]/span[2]').text,
            "item_season": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[2]/div[2]/ul/li[2]/div[2]/span[1]').text,
            "item_sex": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[2]/div[2]/ul/li[2]/div[2]/span[3]').text,
            "item_price": item_price
        }
            
            driver.back()
        return item_details, div  # 성공 시 아이템 정보와 현재 div 반환

    # except NoSuchElementException:
    #     print(f"No item found at position {item} for brand {brand} on page {page_num}.")
    #     return None, div

    except Exception as e:
        print(f"2An error occurred: {e}")
        driver.back()
        return None, div




def save_data(data, filename):
    if data:
        new_data = pd.DataFrame(data)
        if not os.path.exists(filename):
            new_data.to_csv(filename, index=False)
        else:
            new_data.to_csv(filename, mode='a', header=False, index=False)
    else:
        print("No data to save.")

def scrape_brand_data(driver, brand, completed_brands, start_page=1, end_page=100):
    global current_div  # 전역 변수 사용 선언

    for page_num in range(start_page, end_page + 1):
        page_url = f'https://www.musinsa.com/brands/{brand}?page={page_num}&size=90'
        driver.get(page_url)

        if is_page_empty(driver):
            print(f"No items found for brand {brand} on page {page_num}.")
            break

        data = []
        for item in tqdm(range(1, 91), desc=f"Scraping {brand} - Page {page_num}"):
            item_details, new_div = scrape_product_info(driver, item, current_div, brand, page_num)  # brand 및 page_num 변수 전달
            current_div = new_div  # 전역 변수 current_div를 업데이트
            if item_details:
                data.append(item_details)
            else:  
                print(f"No item found at position {item} for brand {brand} on page {page_num}.")

        save_data(data, 'Items_Info.csv') 
        data.clear()

        completed_brands.append({'Brand': brand, 'Page_Number': page_num}) 
        save_progress(completed_brands)

def main():
    driver = setup_driver()
    df = pd.read_csv('Brand.csv')
    brand_names = df['Brand_ENG'].tolist()
    completed_brands = load_progress()

    for brand in tqdm(brand_names, desc="Overall Progress"):
        last_completed_page = next((item['Page_Number'] for item in completed_brands if item['Brand'] == brand), 0)
        start_page = last_completed_page + 1
        print(f"Starting scraping for brand: {brand} from page {start_page}")
        scrape_brand_data(driver, brand, completed_brands, start_page=start_page)

    driver.quit()

if __name__ == "__main__":
    main()

Overall Progress:   0%|          | 0/2929 [00:00<?, ?it/s]

Starting scraping for brand: MUSINSASTANDARD from page 2


Scraping MUSINSASTANDARD - Page 2: 100%|██████████| 90/90 [02:36<00:00,  1.74s/it]


2An error occurred: local variable 'item_element_xpath' referenced before assignment
No item found at position 79 for brand MUSINSASTANDARD on page 2.
2An error occurred: local variable 'item_element_xpath' referenced before assignment
No item found at position 80 for brand MUSINSASTANDARD on page 2.
2An error occurred: local variable 'item_element_xpath' referenced before assignment
No item found at position 81 for brand MUSINSASTANDARD on page 2.
2An error occurred: local variable 'item_element_xpath' referenced before assignment
No item found at position 82 for brand MUSINSASTANDARD on page 2.
2An error occurred: local variable 'item_element_xpath' referenced before assignment
No item found at position 83 for brand MUSINSASTANDARD on page 2.
2An error occurred: local variable 'item_element_xpath' referenced before assignment
No item found at position 84 for brand MUSINSASTANDARD on page 2.
2An error occurred: local variable 'item_element_xpath' referenced before assignment
No item f

: 

In [1]:
import os
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from tqdm import tqdm



def setup_driver():
    options = webdriver.ChromeOptions()
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument("--no-sandbox")
    driver = webdriver.Chrome(options=options)
    driver.set_window_size(1800, 1800)
    return driver

def load_progress(filename='completed_brands.csv'):
    try:
        return pd.read_csv(filename).to_dict('records')
    except FileNotFoundError:
        return []

def save_progress(completed_brands, filename='completed_brands.csv'):
    pd.DataFrame(completed_brands, columns=['Brand', 'Page_Number']).to_csv(filename, index=False)

def is_page_empty(driver):
    try:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="searchList"]/li[1]')))
        return False
    except TimeoutException:
        try:
            no_items_text = driver.find_element(By.XPATH, '//*[@id="result-none-area"]/p').text
            if "등록된 상품이 없습니다" in no_items_text:
                return True
        except NoSuchElementException:
            pass
        return False

def scrape_product_info(driver, item):
    try:
        
        try:
            time.sleep(1)
            item_price_xpath = f'//*[@id="searchList"]/li[{item}]/div[3]/div[2]/p[3]'
            item_price_element = driver.find_element(By.XPATH, item_price_xpath)
        except NoSuchElementException:
            time.sleep(1)
            item_price_xpath = f'//*[@id="searchList"]/li[{item}]/div[4]/div[2]/p[3]'
            item_price_element = driver.find_element(By.XPATH, item_price_xpath)

        item_price = item_price_element.text

        try:
            item_element_xpath = f'//*[@id="searchList"]/li[{item}]/div[3]/div[2]/p[2]/a'
            item_element = driver.find_element(By.XPATH, item_element_xpath)
            driver.execute_script("arguments[0].click();", item_element)
            time.sleep(1)
        except NoSuchElementException:
            item_element_xpath = f'//*[@id="searchList"]/li[{item}]/div[4]/div[2]/p[2]/a'
            item_element = driver.find_element(By.XPATH, item_element_xpath)
            driver.execute_script("arguments[0].click();", item_element)
            time.sleep(1)
           


        try:
            see_button = driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[2]/div[1]/div[4]/button')
            see_button.click()
            time.sleep(1)
        except NoSuchElementException:
            pass

        try:
            item_details = {
            "item_category": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[1]/div[1]/a[1]').text,
            "item_img": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[1]/div[1]/div/img').get_attribute('src'),
            "item_name": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[1]/div[3]/div[2]/h3').text,
            "item_num": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[2]/div[2]/ul/li[1]/div[2]/span[2]').text,
            "item_season": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[2]/div[2]/ul/li[2]/div[2]/span[1]').text,
            "item_sex": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[2]/div[2]/ul/li[2]/div[2]/span[3]').text,
            "item_price": item_price
        }
        except NoSuchElementException:
            item_details = {
            "item_category": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[1]/div[1]/a[1]').text,
            "item_img": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[1]/div[1]/div/img').get_attribute('src'),
            "item_name": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[1]/div[3]/div[2]/h3').text,
            "item_num": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[2]/div[2]/ul/li[1]/div[2]/span[2]').text,
            "gender": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[2]/div[2]/ul/li[2]/div[2]/span').text,
            "item_price": item_price
        }

        driver.back()
        time.sleep(1)
        return item_details  # 성공 시 아이템 정보와 현재 div 반환

    except Exception as e:
        print(f"An error occurred: {e}")
        driver.back()
        return None

def save_data(data, filename):
    if data:
        new_data = pd.DataFrame(data)
        if not os.path.exists(filename):
            new_data.to_csv(filename, index=False)
        else:
            new_data.to_csv(filename, mode='a', header=False, index=False)
    else:
        print("No data to save.")

def scrape_brand_data(driver, brand, completed_brands, start_page=1, end_page=100):


    for page_num in range(start_page, end_page + 1):
        page_url = f'https://www.musinsa.com/brands/{brand}?page={page_num}&size=90'
        driver.get(page_url)

        if is_page_empty(driver):
            print(f"No items found for brand {brand} on page {page_num}.")
            break

        data = []
        for item in tqdm(range(42, 91), desc=f"Scraping {brand} - Page {page_num}"):
            item_details = scrape_product_info(driver, item)
            if item_details:
                data.append(item_details)
                
            else:  
                print(f"No item found at position {item} for brand {brand} on page {page_num}.")

        save_data(data, 'Items_Info.csv') 
        data.clear()

        completed_brands.append({'Brand': brand, 'Page_Number': page_num}) 
        save_progress(completed_brands)

def main():
    driver = setup_driver()
    df = pd.read_csv('Brand.csv')
    brand_names = df['Brand_ENG'].tolist()
    completed_brands = load_progress()

    for brand in tqdm(brand_names, desc="Overall Progress"):
        last_completed_page = next((item['Page_Number'] for item in completed_brands if item['Brand'] == brand), 0)
        start_page = last_completed_page + 1
        print(f"Starting scraping for brand: {brand} from page {start_page}")
        scrape_brand_data(driver, brand, completed_brands, start_page=start_page)

    driver.quit()

if __name__ == "__main__":
    main()

Overall Progress:   0%|          | 0/2929 [00:00<?, ?it/s]

Starting scraping for brand: MUSINSASTANDARD from page 1


Scraping MUSINSASTANDARD - Page 1: 100%|██████████| 49/49 [04:27<00:00,  5.45s/it]


: 

In [ ]:
import os
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from tqdm import tqdm



def setup_driver():
    options = webdriver.ChromeOptions()
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument("--no-sandbox")
    driver = webdriver.Chrome(options=options)
    driver.set_window_size(1800, 1800)
    return driver

def load_progress(filename='completed_brands.csv'):
    try:
        return pd.read_csv(filename).to_dict('records')
    except FileNotFoundError:
        return []

def save_progress(completed_brands, filename='completed_brands.csv'):
    pd.DataFrame(completed_brands, columns=['Brand', 'Page_Number']).to_csv(filename, index=False)

def is_page_empty(driver):
    try:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="searchList"]/li[1]')))
        return False
    except TimeoutException:
        try:
            no_items_text = driver.find_element(By.XPATH, '//*[@id="result-none-area"]/p').text
            if "등록된 상품이 없습니다" in no_items_text:
                return True
        except NoSuchElementException:
            pass
        return False

def scrape_product_info(driver, item):
    try:
        
        try:
         
            item_price_xpath = f'//*[@id="searchList"]/li[{item}]/div[3]/div[2]/p[3]'
            item_price_element = driver.find_element(By.XPATH, item_price_xpath)
        except NoSuchElementException:
          
            item_price_xpath = f'//*[@id="searchList"]/li[{item}]/div[4]/div[2]/p[3]'
            item_price_element = driver.find_element(By.XPATH, item_price_xpath)

        item_price = item_price_element.text

        try:
            item_element_xpath = f'//*[@id="searchList"]/li[{item}]/div[3]/div[2]/p[2]/a'
            item_element = driver.find_element(By.XPATH, item_element_xpath)
            driver.execute_script("arguments[0].click();", item_element)
         
        except NoSuchElementException:
            item_element_xpath = f'//*[@id="searchList"]/li[{item}]/div[4]/div[2]/p[2]/a'
            item_element = driver.find_element(By.XPATH, item_element_xpath)
            driver.execute_script("arguments[0].click();", item_element)
          
           


        try:
            see_button = driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[2]/div[1]/div[4]/button')
            see_button.click()
           
        except NoSuchElementException:
            pass

        try:
            item_details = {
            "item_category": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[1]/div[1]/a[1]').text,
            "item_img": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[1]/div[1]/div/img').get_attribute('src'),
            "item_name": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[1]/div[3]/div[2]/h3').text,
            "item_num": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[2]/div[2]/ul/li[1]/div[2]/span[2]').text,
            "item_season": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[2]/div[2]/ul/li[2]/div[2]/span[1]').text,
            "item_sex": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[2]/div[2]/ul/li[2]/div[2]/span[3]').text,
            "item_price": item_price
        }
        except NoSuchElementException:
            item_details = {
            "item_category": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[1]/div[1]/a[1]').text,
            "item_img": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[1]/div[1]/div/img').get_attribute('src'),
            "item_name": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[1]/div[3]/div[2]/h3').text,
            "item_num": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[2]/div[2]/ul/li[1]/div[2]/span[2]').text,
            "gender": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[2]/div[2]/ul/li[2]/div[2]/span').text,
            "item_price": item_price
        }

        driver.back()
    
        return item_details  # 성공 시 아이템 정보와 현재 div 반환

    except Exception as e:
        print(f"An error occurred: {e}")
        
        return None

def save_data(data, filename):
    if data:
        new_data = pd.DataFrame(data)
        if not os.path.exists(filename):
            new_data.to_csv(filename, index=False)
        else:
            new_data.to_csv(filename, mode='a', header=False, index=False)
    else:
        print("No data to save.")

def scrape_brand_data(driver, brand, completed_brands, start_page=1, end_page=100):


    for page_num in range(start_page, end_page + 1):
        page_url = f'https://www.musinsa.com/brands/{brand}?page={page_num}&size=90'
        driver.get(page_url)

        if is_page_empty(driver):
            print(f"No items found for brand {brand} on page {page_num}.")
            break

        data = []
        for item in tqdm(range(1, 91), desc=f"Scraping {brand} - Page {page_num}"):
            item_details = scrape_product_info(driver, item)
            if item_details:
                data.append(item_details)
                
            else:  
                print(f"No item found at position {item} for brand {brand} on page {page_num}.")

        save_data(data, 'Items_Info.csv') 
        data.clear()

        completed_brands.append({'Brand': brand, 'Page_Number': page_num}) 
        save_progress(completed_brands)

def main():
    driver = setup_driver()
    df = pd.read_csv('Brand.csv')
    brand_names = df['Brand_ENG'].tolist()
    completed_brands = load_progress()

    for brand in tqdm(brand_names, desc="Overall Progress"):
        last_completed_page = next((item['Page_Number'] for item in completed_brands if item['Brand'] == brand), 0)
        start_page = last_completed_page + 1
        print(f"Starting scraping for brand: {brand} from page {start_page}")
        scrape_brand_data(driver, brand, completed_brands, start_page=start_page)

    driver.quit()

if __name__ == "__main__":
    main()

In [ ]:
import os
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from tqdm import tqdm



def setup_driver():
    options = webdriver.ChromeOptions()
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument("--no-sandbox")
    driver = webdriver.Chrome(options=options)
    driver.set_window_size(1800, 1800)
    return driver

def load_progress(filename='completed_brands.csv'):
    try:
        # CSV 파일을 읽어와서 DataFrame으로 변환
        df = pd.read_csv(filename)
        # DataFrame의 가장 마지막 행을 가져와서 브랜드와 페이지 번호 추출
        last_row = df.iloc[-1]
        brand = last_row['Brand']
        last_completed_page = last_row['Page_Number']
        # 브랜드와 페이지 번호를 튜플로 반환
        return brand, last_completed_page
    except FileNotFoundError:
        return None, 0  # 파일이 없는 경우 0을 반환


def save_progress(completed_brands, filename='completed_brands.csv'):
    pd.DataFrame(completed_brands, columns=['Brand', 'Page_Number']).to_csv(filename, index=False)

def is_page_empty(driver):
    try:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="searchList"]/li[1]')))
        return False
    except TimeoutException:
        try:
            no_items_text = driver.find_element(By.XPATH, '//*[@id="result-none-area"]/p').text
            if "등록된 상품이 없습니다" in no_items_text:
                return True
        except NoSuchElementException:
            pass
        return False

def scrape_product_info(driver, item):
    try:
        
        try:
         
            item_price_xpath = f'//*[@id="searchList"]/li[{item}]/div[3]/div[2]/p[3]'
            item_price_element = driver.find_element(By.XPATH, item_price_xpath)
        except NoSuchElementException:
          
            item_price_xpath = f'//*[@id="searchList"]/li[{item}]/div[4]/div[2]/p[3]'
            item_price_element = driver.find_element(By.XPATH, item_price_xpath)

        item_price = item_price_element.text

        try:
            item_element_xpath = f'//*[@id="searchList"]/li[{item}]/div[3]/div[2]/p[2]/a'
            item_element = driver.find_element(By.XPATH, item_element_xpath)
            driver.execute_script("arguments[0].click();", item_element)
         
        except NoSuchElementException:
            item_element_xpath = f'//*[@id="searchList"]/li[{item}]/div[4]/div[2]/p[2]/a'
            item_element = driver.find_element(By.XPATH, item_element_xpath)
            driver.execute_script("arguments[0].click();", item_element)
          
           


        try:
            see_button = driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[2]/div[1]/div[4]/button')
            see_button.click()
           
        except NoSuchElementException:
            pass

        try:
            item_details = {
            "item_category": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[1]/div[1]/a[1]').text,
            "item_img": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[1]/div[1]/div/img').get_attribute('src'),
            "item_name": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[1]/div[3]/div[2]/h3').text,
            "item_num": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[2]/div[2]/ul/li[1]/div[2]/span[2]').text,
            # "item_season": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[2]/div[2]/ul/li[2]/div[2]/span[1]').text,
            # "item_sex": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[2]/div[2]/ul/li[2]/div[2]/span[3]').text,
            "item_price": item_price
        }
        except NoSuchElementException:
            item_details = {
            "item_category": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[1]/div[1]/a[1]').text,
            "item_img": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[1]/div[1]/div/img').get_attribute('src'),
            "item_name": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[1]/div[3]/div[2]/h3').text,
            "item_num": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[2]/div[2]/ul/li[1]/div[2]/span[2]').text,
            # "gender": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[2]/div[2]/ul/li[2]/div[2]/span').text,
            "item_price": item_price
        }

        driver.back()
    
        return item_details  # 성공 시 아이템 정보와 현재 div 반환

    except Exception as e:
        print(f"An error occurred: {e}")
        
        return None

def save_data(data, filename):
    if data:
        new_data = pd.DataFrame(data)
        if not os.path.exists(filename):
            new_data.to_csv(filename, index=False)
        else:
            new_data.to_csv(filename, mode='a', header=False, index=False)
    else:
        print("No data to save.")

def scrape_brand_data(driver, brand, completed_brands, start_page=1, end_page=100):


    for page_num in range(start_page, end_page + 1):
        page_url = f'https://www.musinsa.com/brands/{brand}?page={page_num}&size=90'
        driver.get(page_url)

        if is_page_empty(driver):
            print(f"No items found for brand {brand} on page {page_num}.")
            break

        data = []
        for item in tqdm(range(1, 91), desc=f"Scraping {brand} - Page {page_num}"):
            item_details = scrape_product_info(driver, item)
            if item_details:
                data.append(item_details)
                
            else:  
                print(f"No item found at position {item} for brand {brand} on page {page_num}.")

        save_data(data, 'Items_Info.csv') 
        data.clear()

        completed_brands.append({'Brand': brand, 'Page_Number': page_num}) 
        save_progress(completed_brands)

def main():
    driver = setup_driver()
    df = pd.read_csv('Brand.csv')
    brand_names = df['Brand_ENG'].tolist()
    last_completed_brand, start_page = load_progress()

    for brand in tqdm(brand_names, desc="Overall Progress"):
        if brand == last_completed_brand:
            start_page += 1
        print(f"Starting scraping for brand: {brand} from page {start_page}")
        scrape_brand_data(driver, brand, [], start_page=start_page)  # completed_brands 매개변수 추가
        start_page = 1  # 다음 브랜드를 위해 시작 페이지 번호 초기화

    driver.quit()

if __name__ == "__main__":
    main()

계속 해왔떤것

In [7]:
import os
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from tqdm import tqdm



def setup_driver():
    options = webdriver.ChromeOptions()
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument("--no-sandbox")
    driver = webdriver.Chrome(options=options)
    driver.set_window_size(1800, 1800)
    return driver

def load_progress(filename='completed_brands.csv'):
    try:
        df = pd.read_csv(filename)
        if df.empty:
            return None, 1  # 비어 있는 파일일 경우 None, 0 반환
        last_row = df.iloc[-1]
        brand = last_row['Brand']
        last_completed_page = last_row['Page_Number']
        return brand, last_completed_page
    except FileNotFoundError:
        return None, 1


def save_progress(completed_brands, filename='completed_brands.csv'):
    pd.DataFrame(completed_brands, columns=['Brand', 'Page_Number']).to_csv(filename, index=False)

def is_page_empty(driver):
    try:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="searchList"]/li[1]')))
        return False
    except TimeoutException:
        try:
            no_items_text = driver.find_element(By.XPATH, '//*[@id="result-none-area"]/p').text
            if "등록된 상품이 없습니다" in no_items_text:
                return True
        except NoSuchElementException:
            pass
        return False

def scrape_product_info(driver, brand, item):
    try:
        try:
            item_price_xpath = f'//*[@id="searchList"]/li[{item}]/div[3]/div[2]/p[3]'
            item_price_element = driver.find_element(By.XPATH, item_price_xpath)
            item_price = item_price_element.text
            print("1")
        except NoSuchElementException:
            try:
                item_price_xpath = f'//*[@id="searchList"]/li[{item}]/div[4]/div[2]/p[3]'
                item_price_element = driver.find_element(By.XPATH, item_price_xpath)
                item_price = item_price_element.text
                print("2")
            except NoSuchElementException:
                try:
                    item_price_xpath = f'//*[@id="searchList"]/li[{item}]/div[2]/div[2]/p[3]'
                    item_price_element = driver.find_element(By.XPATH, item_price_xpath)
                    item_price = item_price_element.text
                    print("3")
                except NoSuchElementException:
                    try:
                        item_price_xpath = f'//*[@id="searchList"]/li[{item}]/div[1]/div[2]/p[3]'
                        item_price_element = driver.find_element(By.XPATH, item_price_xpath)
                        item_price = item_price_element.text
                        print("4")
                    except NoSuchElementException:
                        item_price = None

        try:
            item_element_xpath = f'//*[@id="searchList"]/li[{item}]/div[3]/div[2]/p[2]/a'
            item_element = driver.find_element(By.XPATH, item_element_xpath)
            driver.execute_script("arguments[0].click();", item_element)
        
        except NoSuchElementException:
            item_element_xpath = f'//*[@id="searchList"]/li[{item}]/div[4]/div[2]/p[2]/a'
            item_element = driver.find_element(By.XPATH, item_element_xpath)
            driver.execute_script("arguments[0].click();", item_element)
            

        try:
            item_details = {
                "brand": brand,
                "item_category": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[1]/div[1]/a[1]').text,
                "item_img": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[1]/div[1]/div/img').get_attribute('src'),
                "item_name": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[1]/div[3]/div[2]/h3').text,
                "item_num": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[2]/div[2]/ul/li[1]/div[2]/span[2]').text,
                "item_brand": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[2]/div[2]/ul/li[1]/div[2]/a').text,
                "item_price": item_price
            }
            driver.back()
        except NoSuchElementException:
            item_details = {
                "brand": brand,
                "item_category": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[1]/div[1]/a[1]').text,
                "item_img": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[1]/div[1]/div/img').get_attribute('src'),
                "item_name": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[1]/div[3]/div[2]/h3').text,
                "item_num": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[2]/div[1]/ul/li[1]/div[2]/span[2]').text,
                "item_brand": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[2]/div[1]/ul/li[1]/div[2]/a').text,
                "item_price": item_price
            }
         
            driver.back()

        return item_details 
    except NoSuchElementException:
        driver.back()
        return None

def save_data(data, brand, page_num, brand_page_info='Items_Info.csv'):
    if data:
        new_data = pd.DataFrame(data)
        brand_page_info = f"./Datasets/{brand}_page_{page_num}.csv"  # 새로운 경로 설정
        if not os.path.exists("./Datasets"):  # 디렉토리가 없으면 생성
            os.makedirs("./Datasets")
        if not os.path.exists(brand_page_info):
            new_data.to_csv(brand_page_info, index=False)
        else:
            new_data.to_csv(brand_page_info, mode='a', header=False, index=False)
    else:
        print("No data to save.")

def scrape_brand_data(driver, brand, completed_brands, start_page=1, end_page=100):
    for page_num in range(start_page, end_page + 1):
        page_url = f'https://www.musinsa.com/brands/{brand}?page={page_num}&size=90'
        driver.get(page_url)

        if is_page_empty(driver):
            print(f"No items found for brand {brand} on page {page_num}.")
            break

        data = []
        for item in tqdm(range(1, 91), desc=f"Scraping {brand} - Page {page_num}"):
            item_details = scrape_product_info(driver, brand, item) 
            if item_details:
                data.append(item_details)
            else:
                print(f"No item found at position {item} for brand {brand} on page {page_num}.")

        save_data(data, brand, page_num)
        data.clear()

        completed_brands.append({'Brand': brand, 'Page_Number': page_num})
        save_progress(completed_brands)

def main():
    driver = setup_driver()
    df = pd.read_csv('Brand_Name.csv')
    brand_names = df['Brand_ENG'].tolist()
    last_completed_brand, start_page = load_progress()

    for brand in tqdm(brand_names, desc="Overall Progress"):
        if brand == last_completed_brand:
            start_page += 1
        print(f"Starting scraping for brand: {brand} from page {start_page}")
        scrape_brand_data(driver, brand, [], start_page=start_page)  # completed_brands 매개변수 추가
        start_page = 1  # 다음 브랜드를 위해 시작 페이지 번호 초기화

    driver.quit()

if __name__ == "__main__":
    main()

Overall Progress:   0%|          | 0/2910 [00:00<?, ?it/s]

Starting scraping for brand: COVERNAT from page 1


1


1


1


...

No item found at position 2 for brand COVERNAT on page 2.
1


3
No item found at position 4 for brand COVERNAT on page 2.
No item found at position 5 for brand COVERNAT on page 2.
No item found at position 6 for brand COVERNAT on page 2.
No item found at position 7 for brand COVERNAT on page 2.


No item found at position 8 for brand COVERNAT on page 2.
No item found at position 9 for brand COVERNAT on page 2.
No item found at position 10 for brand COVERNAT on page 2.
No item found at position 11 for brand COVERNAT on page 2.
No item found at position 12 for brand COVERNAT on page 2.
No item found at position 13 for brand COVERNAT on page 2.


Overall Progress:   0%|          | 0/2910 [00:29<?, ?it/s]


KeyboardInterrupt: 

수정해본것 1,2,3,4 로


In [ ]:
import os
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from tqdm import tqdm



def setup_driver():
    options = webdriver.ChromeOptions()
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument("--no-sandbox")
    driver = webdriver.Chrome(options=options)
    driver.set_window_size(1800, 1800)
    return driver

def load_progress(filename='completed_brands.csv'):
    try:
        df = pd.read_csv(filename)
        if df.empty:
            return None, 1  # 비어 있는 파일일 경우 None, 0 반환
        last_row = df.iloc[-1]
        brand = last_row['Brand']
        last_completed_page = last_row['Page_Number']
        return brand, last_completed_page
    except FileNotFoundError:
        return None, 1


def save_progress(completed_brands, filename='completed_brands.csv'):
    pd.DataFrame(completed_brands, columns=['Brand', 'Page_Number']).to_csv(filename, index=False)

def is_page_empty(driver):
    try:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="searchList"]/li[1]')))
        return False
    except TimeoutException:
        try:
            no_items_text = driver.find_element(By.XPATH, '//*[@id="result-none-area"]/p').text
            if "등록된 상품이 없습니다" in no_items_text:
                return True
        except NoSuchElementException:
            pass
        return False

def scrape_product_info(driver, brand, item):
    try:
        WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, f'//*[@id="searchList"]/li[{item}]/div[3]/div[2]/p[3]')))
        item_price_element = driver.find_element(By.XPATH, f'//*[@id="searchList"]/li[{item}]/div[3]/div[2]/p[3]')
        item_price = item_price_element.text
    except NoSuchElementException:
        try:
            WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, f'//*[@id="searchList"]/li[{item}]/div[4]/div[2]/p[3]')))
            item_price_element = driver.find_element(By.XPATH, f'//*[@id="searchList"]/li[{item}]/div[4]/div[2]/p[3]')
            item_price = item_price_element.text
        except NoSuchElementException:
            try:
                WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, f'//*[@id="searchList"]/li[{item}]/div[2]/div[2]/p[3]')))
                item_price_element = driver.find_element(By.XPATH, f'//*[@id="searchList"]/li[{item}]/div[2]/div[2]/p[3]')
                item_price = item_price_element.text
            except NoSuchElementException:
                try:
                    WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, f'//*[@id="searchList"]/li[{item}]/div[1]/div[2]/p[3]')))
                    item_price_element = driver.find_element(By.XPATH, f'//*[@id="searchList"]/li[{item}]/div[1]/div[2]/p[3]')
                    item_price = item_price_element.text
                except NoSuchElementException:
                    item_price = None

    try:
        WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, f'//*[@id="searchList"]/li[{item}]/div[3]/div[2]/p[2]/a')))
        item_element = driver.find_element(By.XPATH, f'//*[@id="searchList"]/li[{item}]/div[3]/div[2]/p[2]/a')
        driver.execute_script("arguments[0].click();", item_element)
    except NoSuchElementException:
        try:
            WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, f'//*[@id="searchList"]/li[{item}]/div[4]/div[2]/p[2]/a')))
            item_element = driver.find_element(By.XPATH, f'//*[@id="searchList"]/li[{item}]/div[4]/div[2]/p[2]/a')
            driver.execute_script("arguments[0].click();", item_element)
        except NoSuchElementException:
            try:
                WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, f'//*[@id="searchList"]/li[{item}]/div[2]/div[2]/p[2]/a')))
                item_element = driver.find_element(By.XPATH, f'//*[@id="searchList"]/li[{item}]/div[2]/div[2]/p[2]/a')
                driver.execute_script("arguments[0].click();", item_element)
            except NoSuchElementException:
                try:
                    WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, f'//*[@id="searchList"]/li[{item}]/div[1]/div[2]/p[2]/a')))
                    item_element = driver.find_element(By.XPATH, f'//*[@id="searchList"]/li[{item}]/div[1]/div[2]/p[2]/a')
                    driver.execute_script("arguments[0].click();", item_element)
                except NoSuchElementException:
                    pass

        try:
            item_details = {
                "brand": brand,
                "item_category": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[1]/div[1]/a[1]').text,
                "item_img": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[1]/div[1]/div/img').get_attribute('src'),
                "item_name": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[1]/div[3]/div[2]/h3').text,
                "item_num": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[2]/div[2]/ul/li[1]/div[2]/span[2]').text,
                "item_brand": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[2]/div[2]/ul/li[1]/div[2]/a').text,
                "item_price": item_price
            }

            
        except NoSuchElementException:
            try:
                item_details = {
                "brand": brand,
                "item_category": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[1]/div[1]/a[1]').text,
                "item_img": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[1]/div[1]/div/img').get_attribute('src'),
                "item_name": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[1]/div[3]/div[2]/h3').text,
                "item_num": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[2]/div[1]/ul/li[1]/div[2]/span[2]').text,
                "item_brand": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[2]/div[2]/ul/li[1]/div[2]/a').text,
                "item_price": item_price
            }
            except NoSuchElementException:
                try:
                    item_details = {
                    "brand": brand,
                    "item_category": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[1]/div[1]/a[1]').text,
                    "item_img": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[1]/div[1]/div/img').get_attribute('src'),
                    "item_name": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[1]/div[3]/div[2]/h3').text,
                    "item_num": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[2]/div[1]/ul/li[1]/div[2]/span[2]').text,
                    "item_brand": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[2]/div[1]/ul/li[1]/div[2]/a').text,
                    "item_price": item_price
                }
                except NoSuchElementException:
                    
                    item_details = {
                    "brand": brand,
                    "item_category": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[1]/div[1]/a[1]').text,
                    "item_img": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[1]/div[1]/div/img').get_attribute('src'),
                    "item_name": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[1]/div[3]/div[2]/h3').text,
                    "item_num": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[2]/div[2]/ul/li[1]/div[2]/span[2]').text,
                    "item_brand": driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[2]/div[1]/ul/li[1]/div[2]/a').text,
                    "item_price": item_price
                }
                driver.back()

        return item_details 
    except NoSuchElementException:
        driver.back()
        return None

def save_data(data, brand, page_num, brand_page_info='Items_Info.csv'):
    if data:
        new_data = pd.DataFrame(data)
        brand_page_info = f"./Datasets/{brand}_page_{page_num}.csv"  # 새로운 경로 설정
        if not os.path.exists("./Datasets"):  # 디렉토리가 없으면 생성
            os.makedirs("./Datasets")
        if not os.path.exists(brand_page_info):
            new_data.to_csv(brand_page_info, index=False)
        else:
            new_data.to_csv(brand_page_info, mode='a', header=False, index=False)
    else:
        print("No data to save.")

def scrape_brand_data(driver, brand, completed_brands, start_page=1, end_page=100):
    for page_num in range(start_page, end_page + 1):
        page_url = f'https://www.musinsa.com/brands/{brand}?page={page_num}&size=90'
        driver.get(page_url)

        if is_page_empty(driver):
            print(f"No items found for brand {brand} on page {page_num}.")
            break

        data = []
        for item in tqdm(range(1, 91), desc=f"Scraping {brand} - Page {page_num}"):
            item_details = scrape_product_info(driver, brand, item) 
            if item_details:
                data.append(item_details)
            else:
                print(f"No item found at position {item} for brand {brand} on page {page_num}.")

        save_data(data, brand, page_num)
        data.clear()

        completed_brands.append({'Brand': brand, 'Page_Number': page_num})
        save_progress(completed_brands)

def main():
    driver = setup_driver()
    df = pd.read_csv('Brand_Name.csv')
    brand_names = df['Brand_ENG'].tolist()
    last_completed_brand, start_page = load_progress()

    for brand in tqdm(brand_names, desc="Overall Progress"):
        if brand == last_completed_brand:
            start_page += 1
        print(f"Starting scraping for brand: {brand} from page {start_page}")
        scrape_brand_data(driver, brand, [], start_page=start_page)  # completed_brands 매개변수 추가
        start_page = 1  # 다음 브랜드를 위해 시작 페이지 번호 초기화

    driver.quit()

if __name__ == "__main__":
    main()

가독성 높이고 다시


In [11]:
import os
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from tqdm import tqdm

def setup_driver():
    options = webdriver.ChromeOptions()
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument("--no-sandbox")
    driver = webdriver.Chrome(options=options)
    driver.set_window_size(1800, 1800)
    return driver

def load_progress(filename='completed_brands.csv'):
    try:
        df = pd.read_csv(filename)
        if df.empty:
            return None, 1
        last_row = df.iloc[-1]
        brand = last_row['Brand']
        last_completed_page = last_row['Page_Number']
        return brand, last_completed_page
    except FileNotFoundError:
        return None, 1

def save_progress(completed_brands, filename='completed_brands.csv'):
    pd.DataFrame(completed_brands, columns=['Brand', 'Page_Number']).to_csv(filename, index=False)

def is_page_empty(driver):
    try:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="searchList"]/li[1]')))
        return False
    except TimeoutException:
        try:
            no_items_text = driver.find_element(By.XPATH, '//*[@id="result-none-area"]/p').text
            if "등록된 상품이 없습니다" in no_items_text:
                return True
        except NoSuchElementException:
            pass
        return False

def scrape_product_info(driver, brand, item):
    item_price = None
    item_details = {
        "brand": brand,
        "item_category": None,
        "item_img": None,
        "item_name": None,
        "item_num": None,
        "item_brand": None,
        "item_price": None
    }

    try:
        item_price_element = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, f'//*[@id="searchList"]/li[{item}]/div[3]/div[2]/p[3]')))
        item_price = item_price_element.text
    except NoSuchElementException:
        try:
            item_price_element = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, f'//*[@id="searchList"]/li[{item}]/div[4]/div[2]/p[3]')))
            item_price = item_price_element.text
        except NoSuchElementException:
            try:
                item_price_element = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, f'//*[@id="searchList"]/li[{item}]/div[2]/div[2]/p[3]')))
                item_price = item_price_element.text
            except NoSuchElementException:
                try:
                    item_price_element = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, f'//*[@id="searchList"]/li[{item}]/div[1]/div[2]/p[3]')))
                    item_price = item_price_element.text
                except NoSuchElementException:
                    pass

    try:
        item_element = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, f'//*[@id="searchList"]/li[{item}]/div[3]/div[2]/p[2]/a')))
        driver.execute_script("arguments[0].click();", item_element)
    except NoSuchElementException:
        try:
            item_element = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, f'//*[@id="searchList"]/li[{item}]/div[4]/div[2]/p[2]/a')))
            driver.execute_script("arguments[0].click();", item_element)
        except NoSuchElementException:
            try:
                item_element = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, f'//*[@id="searchList"]/li[{item}]/div[2]/div[2]/p[2]/a')))
                driver.execute_script("arguments[0].click();", item_element)
            except NoSuchElementException:
                try:
                    item_element = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, f'//*[@id="searchList"]/li[{item}]/div[1]/div[2]/p[2]/a')))
                    driver.execute_script("arguments[0].click();", item_element)
                except NoSuchElementException:
                    pass

    try:
        item_details["item_category"] = driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[1]/div[1]/a[1]').text
        item_details["item_img"] = driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[1]/div[1]/div/img').get_attribute('src')
        item_details["item_name"] = driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[1]/div[3]/div[2]/h3').text
        item_details["item_num"] = driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[2]/div[2]/ul/li[1]/div[2]/span[2]').text
        item_details["item_brand"] = driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[2]/div[2]/ul/li[1]/div[2]/a').text
        item_details["item_price"] = item_price
    except NoSuchElementException:
        try:
            item_details["item_category"] = driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[1]/div[1]/a[1]').text
            item_details["item_img"] = driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[1]/div[1]/div/img').get_attribute('src')
            item_details["item_name"] = driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[1]/div[3]/div[2]/h3').text
            item_details["item_num"] = driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[2]/div[1]/ul/li[1]/div[2]/span[2]').text
            item_details["item_brand"] = driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[2]/div[2]/ul/li[1]/div[2]/a').text
            item_details["item_price"] = item_price
        except NoSuchElementException:
            try:
                item_details["item_category"] = driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[1]/div[1]/a[1]').text
                item_details["item_img"] = driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[1]/div[1]/div/img').get_attribute('src')
                item_details["item_name"] = driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[1]/div[3]/div[2]/h3').text
                item_details["item_num"] = driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[2]/div[1]/ul/li[1]/div[2]/span[2]').text
                item_details["item_brand"] = driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[2]/div[1]/ul/li[1]/div[2]/a').text
                item_details["item_price"] = item_price
            except NoSuchElementException:
                item_details["item_price"] = item_price
                driver.back()
    driver.back()
    return item_details 

def save_data(data, brand, page_num, brand_page_info='Items_Info.csv'):
    if data:
        new_data = pd.DataFrame(data)
        brand_page_info = f"./Datasets/{brand}_page_{page_num}.csv"
        if not os.path.exists("./Datasets"):
            os.makedirs("./Datasets")
        if not os.path.exists(brand_page_info):
            new_data.to_csv(brand_page_info, index=False)
        else:
            new_data.to_csv(brand_page_info, mode='a', header=False, index=False)
    else:
        print("No data to save.")

def scrape_brand_data(driver, brand, completed_brands, start_page=1, end_page=100):
    for page_num in range(start_page, end_page + 1):
        page_url = f'https://www.musinsa.com/brands/{brand}?page={page_num}&size=90'
        driver.get(page_url)

        if is_page_empty(driver):
            print(f"No items found for brand {brand} on page {page_num}.")
            break

        data = []
        for item in tqdm(range(1, 91), desc=f"Scraping {brand} - Page {page_num}"):
            item_details = scrape_product_info(driver, brand, item) 
            if item_details:
                data.append(item_details)
            else:
                print(f"No item found at position {item} for brand {brand} on page {page_num}.")

        save_data(data, brand, page_num)
        data.clear()

        completed_brands.append({'Brand': brand, 'Page_Number': page_num})
        save_progress(completed_brands)

def main():
    driver = setup_driver()
    df = pd.read_csv('Brand_Name.csv')
    brand_names = df['Brand_ENG'].tolist()
    last_completed_brand, start_page = load_progress()

    for brand in tqdm(brand_names, desc="Overall Progress"):
        if brand == last_completed_brand:
            start_page += 1
        print(f"Starting scraping for brand: {brand} from page {start_page}")
        scrape_brand_data(driver, brand, [], start_page=start_page)
        start_page = 1

    driver.quit()

if __name__ == "__main__":
    main()


Overall Progress:   0%|          | 0/2905 [00:00<?, ?it/s]

Starting scraping for brand: DISCOVERYEXPEDITION from page 5


Overall Progress:   0%|          | 0/2905 [00:19<?, ?it/s]


TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6015FAD02+56930]
	(No symbol) [0x00007FF60156F602]
	(No symbol) [0x00007FF6014242E5]
	(No symbol) [0x00007FF6014698ED]
	(No symbol) [0x00007FF601469A2C]
	(No symbol) [0x00007FF6014AA967]
	(No symbol) [0x00007FF60148BCDF]
	(No symbol) [0x00007FF6014A81E2]
	(No symbol) [0x00007FF60148BA43]
	(No symbol) [0x00007FF60145D438]
	(No symbol) [0x00007FF60145E4D1]
	GetHandleVerifier [0x00007FF601976F8D+3711213]
	GetHandleVerifier [0x00007FF6019D04CD+4077101]
	GetHandleVerifier [0x00007FF6019C865F+4044735]
	GetHandleVerifier [0x00007FF601699736+706710]
	(No symbol) [0x00007FF60157B8DF]
	(No symbol) [0x00007FF601576AC4]
	(No symbol) [0x00007FF601576C1C]
	(No symbol) [0x00007FF6015668D4]
	BaseThreadInitThunk [0x00007FF8CDE87344+20]
	RtlUserThreadStart [0x00007FF8CFC426B1+33]


In [ ]:
import os
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from tqdm import tqdm

def setup_driver():
    options = webdriver.ChromeOptions()
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument("--no-sandbox")
    driver = webdriver.Chrome(options=options)
    driver.set_window_size(1800, 1800)
    return driver

def load_progress(filename='completed_brands.csv'):
    try:
        df = pd.read_csv(filename)
        if df.empty:
            return None, 1
        last_row = df.iloc[-1]
        brand = last_row['Brand']
        last_completed_page = last_row['Page_Number']
        return brand, last_completed_page
    except FileNotFoundError:
        return None, 1

def save_progress(completed_brands, filename='completed_brands.csv'):
    pd.DataFrame(completed_brands, columns=['Brand', 'Page_Number']).to_csv(filename, index=False)

def is_page_empty(driver):
    try:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="searchList"]/li[1]')))
        return False
    except TimeoutException:
        try:
            no_items_text = driver.find_element(By.XPATH, '//*[@id="result-none-area"]/p').text
            if "등록된 상품이 없습니다" in no_items_text:
                return True
        except NoSuchElementException:
            pass
        return False

def scrape_product_info(driver, brand, item):
    item_price = None
    item_details = {
        "brand": brand,
        "item_category": None,
        "item_img": None,
        "item_name": None,
        "item_num": None,
        "item_brand": None,
        "item_price": None
    }

    try:
        item_price_element = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, f'//*[@id="searchList"]/li[{item}]/div[3]/div[2]/p[3]')))
        item_price = item_price_element.text
    except NoSuchElementException:
        try:
            item_price_element = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, f'//*[@id="searchList"]/li[{item}]/div[4]/div[2]/p[3]')))
            item_price = item_price_element.text
        except NoSuchElementException:
            try:
                item_price_element = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, f'//*[@id="searchList"]/li[{item}]/div[2]/div[2]/p[3]')))
                item_price = item_price_element.text
            except NoSuchElementException:
                try:
                    item_price_element = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, f'//*[@id="searchList"]/li[{item}]/div[1]/div[2]/p[3]')))
                    item_price = item_price_element.text
                except NoSuchElementException:
                    pass

    try:
        item_element = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, f'//*[@id="searchList"]/li[{item}]/div[3]/div[2]/p[2]/a')))
        driver.execute_script("arguments[0].click();", item_element)
    except NoSuchElementException:
        try:
            item_element = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, f'//*[@id="searchList"]/li[{item}]/div[4]/div[2]/p[2]/a')))
            driver.execute_script("arguments[0].click();", item_element)
        except NoSuchElementException:
            try:
                item_element = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, f'//*[@id="searchList"]/li[{item}]/div[2]/div[2]/p[2]/a')))
                driver.execute_script("arguments[0].click();", item_element)
            except NoSuchElementException:
                try:
                    item_element = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, f'//*[@id="searchList"]/li[{item}]/div[1]/div[2]/p[2]/a')))
                    driver.execute_script("arguments[0].click();", item_element)
                except NoSuchElementException:
                    pass

    try:
        item_details["item_category"] = driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[1]/div[1]/a[1]').text
        item_details["item_img"] = driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[1]/div[1]/div/img').get_attribute('src')
        item_details["item_name"] = driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[1]/div[3]/div[2]/h3').text
        item_details["item_num"] = driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[2]/div[2]/ul/li[1]/div[2]/span[2]').text
        item_details["item_brand"] = driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[2]/div[2]/ul/li[1]/div[2]/a').text
        item_details["item_price"] = item_price
    except NoSuchElementException:
        try:
            item_details["item_category"] = driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[1]/div[1]/a[1]').text
            item_details["item_img"] = driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[1]/div[1]/div/img').get_attribute('src')
            item_details["item_name"] = driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[1]/div[3]/div[2]/h3').text
            item_details["item_num"] = driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[2]/div[1]/ul/li[1]/div[2]/span[2]').text
            item_details["item_brand"] = driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[2]/div[2]/ul/li[1]/div[2]/a').text
            item_details["item_price"] = item_price
        except NoSuchElementException:
            try:
                item_details["item_category"] = driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[1]/div[1]/a[1]').text
                item_details["item_img"] = driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[1]/div[1]/div/img').get_attribute('src')
                item_details["item_name"] = driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[1]/div[3]/div[2]/h3').text
                item_details["item_num"] = driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[2]/div[1]/ul/li[1]/div[2]/span[2]').text
                item_details["item_brand"] = driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[2]/div[1]/ul/li[1]/div[2]/a').text
                item_details["item_price"] = item_price
            except NoSuchElementException:
                item_details["item_price"] = item_price
                driver.back()
    driver.back()
    return item_details 

def save_data(data, brand, page_num, brand_page_info='Items_Info.csv'):
    if data:
        new_data = pd.DataFrame(data)
        brand_page_info = f"./Datasets/{brand}_page_{page_num}.csv"
        if not os.path.exists("./Datasets"):
            os.makedirs("./Datasets")
        if not os.path.exists(brand_page_info):
            new_data.to_csv(brand_page_info, index=False)
        else:
            new_data.to_csv(brand_page_info, mode='a', header=False, index=False)
    else:
        print("No data to save.")

def scrape_brand_data(driver, brand, completed_brands, start_page=1, end_page=100):
    for page_num in range(start_page, end_page + 1):
        page_url = f'https://www.musinsa.com/brands/{brand}?page={page_num}&size=90'
        driver.get(page_url)

        if is_page_empty(driver):
            print(f"No items found for brand {brand} on page {page_num}.")
            break

        data = []
        for item in tqdm(range(1, 91), desc=f"Scraping {brand} - Page {page_num}"):
            item_details = scrape_product_info(driver, brand, item) 
            if item_details:
                data.append(item_details)
            else:
                print(f"No item found at position {item} for brand {brand} on page {page_num}.")

        save_data(data, brand, page_num)
        data.clear()

        completed_brands.append({'Brand': brand, 'Page_Number': page_num})
        save_progress(completed_brands)

def main():
    driver = setup_driver()
    df = pd.read_csv('Brand_Name.csv')
    brand_names = df['Brand_ENG'].tolist()
    last_completed_brand, start_page = load_progress()

    for brand in tqdm(brand_names, desc="Overall Progress"):
        if brand == last_completed_brand:
            start_page += 1
        print(f"Starting scraping for brand: {brand} from page {start_page}")
        scrape_brand_data(driver, brand, [], start_page=start_page)
        start_page = 1

    driver.quit()

if __name__ == "__main__":
    main()
